Importing libraries

In [ ]:
pip install calmap

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objs as go
import calmap

Creating a dataframe from csv file

In [ ]:
df = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
df.head()

Visualizing the cases of Corona in Indian and Foreigners 

In [ ]:
colors = ['blue','green']

indian = df[df['Date']=='28/03/20']['ConfirmedIndianNational'].sum()
foreign = df[df['Date']=='28/03/20']['ConfirmedForeignNational'].sum()
fig = go.Figure(data=[go.Pie(labels=['Indian','Foreigners'],values= [indian,foreign],hole =.3)])
                          
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#FFFFFF', width=2)))
fig.show()

Replacing the columns 'ConfirmedIndianNational' and	'ConfirmedForeignNational' by 'Confirmed'

In [ ]:
df['Confirmed'] =  df['ConfirmedIndianNational'] + df['ConfirmedForeignNational']
df.drop('ConfirmedIndianNational',axis = 1,inplace=True)
df.drop('ConfirmedForeignNational',axis = 1,inplace=True)
df

Creating a column for 'Active' cases

In [ ]:
df['Active'] = df['Confirmed'] - df['Cured'] - df['Deaths']
df

Creating a dataframe with the latest data 

In [ ]:
latest_df = df[df['Date'] == df['Date'][len(df)-1]].reset_index()
latest_df

Visualizing the data based on number of cases

In [ ]:
latest_grouped = latest_df.groupby('State/UnionTerritory')['Cured', 'Deaths', 'Confirmed', 'Active'].sum().reset_index()

In [ ]:
temp = latest_grouped.sort_values(by='Confirmed', ascending=False)
temp = temp[['State/UnionTerritory', 'Confirmed', 'Active', 'Deaths', 'Cured']]
temp['Mortality Rate'] = round((temp['Deaths']/temp['Confirmed'])*100,2)
temp = temp.reset_index(drop=True)

temp.style.background_gradient(cmap="Reds", subset=['Confirmed', 'Active'])\
            .background_gradient(cmap="Greens", subset=['Cured'])\
            .background_gradient(cmap="Oranges_r", subset=['Deaths'])\
            .background_gradient(cmap="seismic_r",subset=['Mortality Rate'])

Visualizing the data related to 'Cured', 'Death' and 'Active' cases with time using "Interactive plots"

In [ ]:
temp1 = df.groupby('Date')['Cured', 'Deaths', 'Active'].sum().reset_index()
temp1 = temp1.melt(id_vars="Date", value_vars=['Cured', 'Deaths', 'Active'],var_name='Cases', value_name='Counts')
temp1.head()

fig = px.area(temp1, x="Date", y="Counts", color='Cases', height=800,title='Covid-19 cases')
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

Contrasting the data of 'Cured' cases with respect to "Confirmed" cases state-wise

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
data = temp[['State/UnionTerritory','Confirmed','Cured','Deaths']]
data.sort_values('Confirmed',ascending=False,inplace=True)
sns.set_color_codes("pastel")
sns.barplot(x="Confirmed", y="State/UnionTerritory", data=data,label="Total", color="r")

sns.set_color_codes("muted")
sns.barplot(x="Cured", y="State/UnionTerritory", data=data,label="Cured", color="g")

ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(ylabel="States / U.T",xlabel="Cases")
sns.despine(left=True, bottom=True)

Visualizing the same data using Pie-Chart

In [ ]:
x=temp['Confirmed']
labels=temp['State/UnionTerritory']
explode=np.zeros(temp.shape[0],)
explode=explode+0.1


plt.figure(figsize=(10,10))
plt.title("Covid report State-wise", fontsize=16)
plt.pie(x, labels=labels, explode=explode,wedgeprops=dict(width=0.5),autopct='%1.1f%%', startangle=0, )
plt.show()

Creating a new dataframe which keeps changing trend of corona virus in India as days progresses

In [ ]:
rise_covid = pd.DataFrame(columns=['Date','Confirmed','Active','Cured','Deaths'])
for i in range(len(df['Date'].unique())):
    rise_covid.loc[i] = [df['Date'].unique()[i],df[df['Date']==df['Date'].unique()[i]]['Confirmed'].sum(),df[df['Date']==df['Date'].unique()[i]]['Active'].sum(),df[df['Date']==df['Date'].unique()[i]]['Cured'].sum(),df[df['Date']==df['Date'].unique()[i]]['Deaths'].sum()]
                

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=rise_covid['Date'], y=rise_covid['Confirmed'],mode='lines+markers',name='Confirmed'))
fig.add_trace(go.Scatter(x=rise_covid['Date'], y=rise_covid['Cured'], mode='lines',name='Cured'))
fig.add_trace(go.Scatter(x=rise_covid['Date'], y=rise_covid['Active'],mode='lines',name='Active'))
fig.add_trace(go.Scatter(x=rise_covid['Date'], y=rise_covid['Deaths'],mode='lines',name='Deaths'))
        
    
fig.update_layout(title_text='Coronavirus Cases in India',plot_bgcolor='rgb(250, 242, 242)')

fig.show()
